# Test the MinimalMedia method

In [1]:
from modelseedpy.fbapkg.reactionusepkg import MinimalMedia

# define the environment path 
from pprint import pprint
from json import dump
import os
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

# import the models
import cobrakbase
token = 'WNIJW5W2A376KPOGSMCJB4RTEFUULXZE'
kbase_api = cobrakbase.KBaseAPI(token)
model = kbase_api.get_from_ws("E_iAH991V2",40576)

# prevent excessive warnings
import warnings
warnings.filterwarnings(action='once')

C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


cobrakbase 0.2.8


In [2]:
print(model.id)
model2 = model.copy()
print(model2.id)

Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23
Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23


## Test the recursive logic

In [8]:
from optlang import Variable, Constraint
from modelseedpy import FBAHelper
from optlang.symbolics import Zero

def _load_model(org_model, var_types=None):
    print("first", org_model.id)
    model = org_model.copy()
    print(model.id)
    var_types = var_types or ["ru"]
    variables = {var_type:{} for var_type in var_types}
    return model, variables

def _add_constraint(model, constraint, coef=None):
    model.add_cons_vars(constraint)
    if coef:
        constraint.set_linear_coefficients(coef)
    print(constraint)
    model.solver.update()

def minimize_components(org_model):
    """minimize the quantity of metabolites that are consumed by the model"""
    model, variables = _load_model(org_model)
    print(model.id)
    # add a constraint of minimal growth
    _add_constraint(model, Constraint(sum([rxn.flux_expression for rxn in model.reactions if "bio" in rxn.id]),
                              lb=0.1, ub=None, name="min_growth"))

    # define the binary variable and constraint
    for ex_rxn in FBAHelper.exchange_reactions(model):
        # define the variable
        variables["ru"][ex_rxn.id] = Variable(ex_rxn.id+"_ru", lb=0, ub=1, type="binary")
        model.add_cons_vars(variables["ru"])
        # bin_flux: {rxn_bin}*1000 >= {rxn_rev_flux}
        _add_constraint(model, Constraint(Zero, lb=0, ub=None, name=ex_rxn.id+"_bin"),
                                        coef={variables["ru"][ex_rxn.id]: 1000, ex_rxn.reverse_variable: -1})
    print("to objective")
    FBAHelper.add_objective(model, sum([var for var in variables["ru"].values()]), "min")

    # determine each solution
    solutions = []
    sol = model.optimize()
    while sol.status == "optimal":
        solutions.append(sol)
        ## omit the solution from the next search
        _add_constraint(
            model, Constraint(Zero, lb=len(sol_dict) - 1, ub=len(sol_dict) - 1, name=ex_rxn.id + f"_exclusion_sol{len(solutions)}"),
            FBAHelper.solution_to_variables_dict(sol, model))
        sol = model.optimize()

In [ ]:
minimize_components(model)

first Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23


In [6]:
from sys import getsizeof
getsizeof(model.solver.to_lp())

345810

In [ ]:
from optlang import Variable, Constraint
from modelseedpy.core.fbahelper import FBAHelper
from optlang.symbolics import Zero

def _load_model(org_model, var_types=None):
    print("first", org_model.id)
    model = org_model.copy()
    print(model.id)
    var_types = var_types or ["ru"]
    variables = {var_type:{} for var_type in var_types}
    return model, variables

def _add_constraint(model, constraint, coef=None):
    model.add_cons_vars(constraint)
    if coef:
        constraint.set_linear_coefficients(coef)
    print(constraint)
    model.solver.update()

def minimize_components(org_model):
    """minimize the quantity of metabolites that are consumed by the model"""
    model, variables = _load_model(org_model)
    print(model.id)
    # add a constraint of minimal growth
    _add_constraint(model, Constraint(sum([rxn.flux_expression for rxn in model.reactions if "bio" in rxn.id]),
                              lb=0.1, ub=None, name="min_growth"))

    # define the binary variable and constraint
    for ex_rxn in FBAHelper.exchange_reactions(model):
        # define the variable
        variables["ru"][ex_rxn.id] = Variable(ex_rxn.id+"_ru", lb=0, ub=1, type="binary")
        model.add_cons_vars(variables["ru"])
        # bin_flux: {rxn_bin}*1000 >= {rxn_rev_flux}
        _add_constraint(model, Constraint(Zero, lb=0, ub=None, name=ex_rxn.id+"_bin"),
                                        coef={variables["ru"][ex_rxn.id]: 1000, ex_rxn.reverse_variable: -1})
    print("to objective")
    FBAHelper.add_objective(model, sum([var for var in variables["ru"].values()]), "min")
    
minimize_components(model)

first Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23


In [ ]:
%run ../../modelseedpy/modelseedpy/core/minimalmediapkg.py
MinimalMediaPkg.minimize_components(model)

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\minimalmediapkg.py:101: DeprecationWarning: invalid escape sequence \d
  sol_ex_cpds = {re.sub("(_[a-z]\d+)", "", rxn.id).replace("EX_", ""): flux


In [ ]:
from optlang.symbolics import Zero
from modelseedpy import FBAHelper

# add a constraint of minimal growth
cons = model.problem.Constraint(sum([rxn.flux_expression for rxn in model.reactions if "bio" in rxn.id]), lb=0.1, ub=None, name="min_growth")
model.add_cons_vars(cons)
model.solver.update()

# define the exchange variables and constraints
variables = {}
for ex_rxn in FBAHelper.exchange_reactions(model):
    if ex_rxn.lower_bound >= 0:
        print(ex_rxn.bounds)
        
    variables[ex_rxn.id] = model.problem.Variable(ex_rxn.id+"_bin", lb=0, ub=1, type="binary")
    model.add_cons_vars(variables[ex_rxn.id])
    
    # bin_flux: {rxn_bin}*1000 >= {rxn_rev_flux}
    cons = model.problem.Constraint(Zero, lb=0, ub=None, name=ex_rxn.id+"_bin")
    model.add_cons_vars(cons)
    cons.set_linear_coefficients({variables[ex_rxn.id]:1000, ex_rxn.reverse_variable:-1})
    model.solver.update()
FBAHelper.add_objective(model, sum([var for var in variables.values()]), "min")

solutions = []
sol = model.optimize()
while sol.status == "optimal":
    solutions.append(sol)
    sol_dict = FBAHelper.solution_to_variables_dict(sol, model)
    
    ## omit the solution from the next search
    cons = model.problem.Constraint(Zero, lb=len(sol_dict)-1, ub=len(sol_dict)-1, name=ex_rxn.id+f"_exclusion_sol{len(solutions)}")
    model.add_cons_vars(cons)
    model.solver.update()
    cons.set_linear_coefficients(sol_dict)
    
    sol = model.optimize()

In [15]:
print(len(FBAHelper.solution_to_variables_dict(sol, model)))

for k,v in sol.fluxes.items():
    print(k,v)

1530
12ETHDt_c0 -0.0018671999998142452
12PPDRt_c0 0.0
12PPDt_c0 0.0
23PDE2_c0 0.0
23PDE4_c0 0.0
23PDE7_c0 0.0
23PDE9_c0 0.0
26DAPLLATi_c0 -0.04260259999973472
2AMEPHPAT_c0 -0.00046680000000000007
2FUCLAC__DASH__FUCASEe_c0 0.0
2MBCOATA_c0 0.0149072
2MMALD_c0 0.043162200000000005
2MMALD2_c0 0.043162200000000005
3DSPHR_c0 -0.00046679999991038594
3FUCLAC__DASH__FUCASEe_c0 0.0
3HACPR1_c0 0.00046680000000000007
3HACPR2_c0 0.00046680000000000007
3HAD100_c0 0.04037539999991034
3HAD10M11_c0 0.0127776
3HAD10M12_c0 0.0149072
3HAD11M12_c0 0.019263999999910388
3HAD120_c0 0.03611619999991034
3HAD121_c0 0.0042592
3HAD12M13_c0 0.0127776
3HAD12M14_c0 0.0149072
3HAD13M14_c0 0.019263999999910388
3HAD140_c0 0.029911999999910337
3HAD141_c0 0.0042592
3HAD14M15_c0 0.0063888
3HAD14M16_c0 0.0085184
3HAD15M16_c0 0.008985199999910386
3HAD160_c0 0.01750359999991039
3HAD161_c0 0.0042592
3HAD180_c0 0.0085184
3HAD181_c0 0.0042592
3HAD40_c0 0.04037539999991034
3HAD4M5_c0 0.0127776
3HAD4M6_c0 0.0149072
3HAD5M6_c0 0.01

In [4]:
for rxn in sol_dict:
    print(type(rxn), rxn)
    break

<class 'optlang.glpk_interface.Variable'> 0 <= 12ETHDt_c0 <= 1000


In [ ]:
from pprint import pprint
print(solutions)
pprint([(rxn, flux) for rxn, flux in sol_dict.items() if flux < 0])

In [3]:
# from modelseedpy.core.fbahelper import FBAHelper
# from pprint import pprint
# # for rxn in model1.reactions:
# for rxn in FBAHelper.bio_reactions(model1):
#     pprint(dir(rxn))
#     # print(type(rxn.flux_expression))
#     print(type(rxn.forward_variable))
#     break

In [4]:
%run ../../ModelSEEDpy/modelseedpy/fbapkg/reactionusepkg.py
# min_media.pkgmgr.removepkgobj(min_media)
min_media = MinimalMedia(model1)

OptimizationError: Solver status is 'undefined'.

In [ ]:
# # from modelseedpy.core.fbahelper import FBAHelper
# from pprint import pprint
# print(dir(model1.variables))
# # pprint(model1.variables)
# for var, value in model1.variables.items():
#     # pprint(dir(var))
#     print(type(var), var, type(value), value)
#     # print(type(rxn.flux_expression))
#     # print(type(rxn.forward_variable))
#     break

# Construct an Optlang model of minimal media

In [ ]:
# from modelseedpy.core.fbahelper import OptlangHelper

# # construct the model
# variables, constraints = [], []
# for met in media:
#     var_name = f"g_{t}" ; var_bounds = (0, 1000) ; dict_vars.append((var_name, var_bounds, "continuous"))
#     cons_name = f"const_{t}" ; cons_bounds = (-1000, 1000) ; cons_expr = [(t,var_name)]
#     dict_cons.append((cons_name, cons_bounds, cons_expr))

# dict_obj = ("Test Helper Dictionary", [var.name for var in variables], "max")

# dict_model = OptlangHelper.define_model("OptlangHelper test", dict_vars, dict_cons, dict_obj)
# with open("minimal_media_model.json", 'w') as out:
#     dump(dict_model, out, indent=3)
    
# # test the model
# model3 = Model.from_json(dict_model)

# Test Jenga method

In [1]:
import warnings
warnings.filterwarnings(action='once')

# define the environment path 
from pprint import pprint
from json import dump
import os
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

# import the models
import cobrakbase
token = 'RBP7R34IILI6IQN5533VVSVUZM6H7YU7'
kbase_api = cobrakbase.KBaseAPI(token)

model1 = kbase_api.get_from_ws("E_iAH991V2",40576)
model2 = kbase_api.get_from_ws("E_iML1515.kb",40576)
com_model = kbase_api.get_from_ws("CMM_iAH991V2_iML1515.kb",40576)
models = [model1, model2]

C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\io\dict.py:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  value: Union[str, np.float, np.bool, Set, Dict]
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\io\dict.py:71: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecatio

cobrakbase 0.2.8


In [2]:
%run ../../ModelSEEDpy/modelseedpy/community/mscommunity.py
comm_media, jenga_removals = MSCommunity.estimate_minimal_community_media(models, com_model)

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\community\mscommunity.py:213: DeprecationWarning: invalid escape sequence \d
  if re.search('^(bio)(\d+)$', rxn.id):
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Initial media defined with 22 exchanges
Syntrophic fluxes examined after 0.0 minutes, with 2 change(s): {'values_changed': {"root['EX_cpd10515_e0']": {'new_value': 6.8362962026137515, 'old_value': 0.0009336000000000001}, "root['EX_cpd00009_e0']": {'new_value': 2.9779716628444723, 'old_value': 0.17640200000573714}}}
The 40320 permutations of the {'EX_cpd00028_e0', 'EX_cpd00296_e0', 'EX_cpd00104_e0', 'EX_cpd10515_e0', 'EX_cpd00048_e0', 'EX_cpd00009_e0', 'EX_cpd03725_e0', 'EX_cpd00007_e0'} redundant compounds, from absolute tolerance of 1e-4, will be examined.
Unique combinations:
6 [{'EX_cpd00028_e0', 'EX_cpd00296_e0', 'EX_cpd00104_e0', 'EX_cpd10515_e0', 'EX_cpd00048_e0'}, {'EX_cpd00028_e0', 'EX_cpd00296_e0', 'EX_cpd00104_e0', 'EX_cpd00048_e0', 'EX_cpd03725_e0'}, {'EX_cpd00028_e0', 'EX_cpd00296_e0', 'EX_cpd00104_e0', 'EX_cpd10515_e0', 'EX_cpd00007_e0'}, {'EX_cpd00028_e0', 'EX_cpd00104_e0', 'EX_cpd10515_e0', 'EX_cpd00048_e0', 'EX_cpd00009_e0'}, {'EX_cpd00028_e0', 'EX_cpd00104_e0', 'EX_cpd

In [21]:
# comm_media["community_media"]
import json
with open(com_model.id+"_media.json", 'w') as out:
    json.dump(comm_media, out, indent=3)

In [18]:
display(jenga_removals)
print(list(jenga_removals.values())[0])
changed_quantity = len(list(jenga_removals.values())[0])
print(changed_quantity)

{'dictionary_item_removed': [root['EX_cpd10515_e0'], root['EX_cpd00028_e0'], root['EX_cpd00296_e0'], root['EX_cpd00104_e0'], root['EX_cpd00048_e0']]}

[root['EX_cpd10515_e0'], root['EX_cpd00028_e0'], root['EX_cpd00296_e0'], root['EX_cpd00104_e0'], root['EX_cpd00048_e0']]
5


In [3]:
display(comm_media, jenga_removals)

{'community_media': {'EX_cpd00007_e0': 1000,
  'EX_sphmyln_bt_e0': 0.0004668000000129604,
  'EX_cpd01017_e0': 824.2636444260851,
  'EX_cpd00149_e0': 0.0004693000000265238,
  'EX_cpd00009_e0': 2.9779716628444723,
  'EX_cpd00166_e0': 0.0004668000000265238,
  'EX_cpd00205_e0': 0.019986099999982798,
  'EX_cpd00254_e0': 0.0013343000000531902,
  'EX_cpd00034_e0': 0.0005009000000265238,
  'EX_cpd00063_e0': 0.0009873000000197862,
  'EX_cpd00971_e0': 0.0004668000001402106,
  'EX_cpd00030_e0': 0.0005359000000265238,
  'EX_cpd00099_e0': 0.0005204999999932625,
  'EX_cpd03725_e0': 657.6688983940277,
  'EX_cpd00058_e0': 7.09e-05,
  'EX_cpd00244_e0': 3.23e-05,
  'EX_cpd11574_e0': 7.000000000000001e-07},
 'members': {'Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23': {'media': {'EX_cpd00007_e0': 1000.0,
    'EX_sphmyln_bt_e0': 0.0004668000000129604,
    'EX_cpd01017_e0': 824.2636444260851,
    'EX_cpd00149_e0': 0.0004668000000265238,
    'EX_cpd10515_e0': 0.0009336000000000001,
    'EX_cpd00009_e0': 0

{'dictionary_item_removed': [root['EX_cpd10515_e0'], root['EX_cpd00028_e0'], root['EX_cpd00296_e0'], root['EX_cpd00104_e0'], root['EX_cpd00048_e0']]}

In [2]:
%run ../../ModelSEEDpy/modelseedpy/community/mscommunity.py
comm_media, jenga_removals = MSCommunity.estimate_minimal_community_media(models, com_model)

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\community\mscommunity.py:213: DeprecationWarning: invalid escape sequence \d
  if re.search('^(bio)(\d+)$', rxn.id):
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()
C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


Initial media defined with 23 exchanges
Syntrophic fluxes examined after 0.0 minutes, with 2 change(s): {'values_changed': {"root['EX_cpd00009_e0']": {'new_value': 2.982600809488827, 'old_value': 0.17640199999437353}, "root['EX_cpd10515_e0']": {'new_value': 6.892064499920052, 'old_value': 0.0009336000000000001}}}
The 362880 permutations of the {'EX_cpd00028_e0', 'EX_cpd00541_e0', 'EX_cpd00294_e0', 'EX_cpd00009_e0', 'EX_cpd03725_e0', 'EX_cpd10515_e0', 'EX_cpd00048_e0', 'EX_cpd00007_e0', 'EX_cpd00104_e0'} redundant compounds, from absolute tolerance of 1e-4, will be examined.


C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\community\mscommunity.py:213: DeprecationWarning: invalid escape sequence \d
  if re.search('^(bio)(\d+)$', rxn.id):


KeyError: 0